<a href="https://colab.research.google.com/github/areejatariq/Data-cleaning-and-visualization-using-pyspark-/blob/main/Assignment1_DSTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing the pyspark

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 199 kB 57.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=e2f2857794baba6621f18390821cbe034523ba9b6f48524f826f53005ef94065
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:

#creating the pyspark session 
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
#checking if the pyspark session is working
spark


In [ ]:
# Reading the data from the directory
df_bristol = spark.read.csv('/content/bristol-air-quality-data.csv', header=True,sep=";")


In [ ]:
df_bristol.show(10)

+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+----+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5| PM2.5|VPM2.5|  CO|      O3| SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+----+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|19.454|   2.9|null|20.40603|null|       null|null|        null|       AURN St Pauls|51.4628294172,-2....|2006-06-15T00:00:...|               

In [ ]:
#taking the mean of the SO2 and than adding that average back into the column

from pyspark.sql.functions import mean
mean_val=df_bristol.select(mean(df_bristol.SO2)).collect()
print(type(mean_val)) #mean_val is a list row object
print('mean value of SO2', mean_val[0][0])
mean_SO2=mean_val[0][0]

#now using mean_SO2 value to fill the nulls in SO2 column
df_bristol=df_bristol.na.fill(value='7.604697174',subset=['SO2'])

<class 'list'>
mean value of SO2 7.604697174396046


In [ ]:
df_bristol.show()

+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5| PM2.5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|19.454|   2.9|null|20.40603|7.604697174|       null|null|        null|       AURN St Pauls|51.4628294172,-2....|2006-06-

In [ ]:
#taking the mean of the temperature and than adding that average back into the column

from pyspark.sql.functions import mean
mean_val_temp=df_bristol.select(mean(df_bristol.Temperature)).collect()
print(type(mean_val_temp)) #mean_val_temp is a list row object
print('mean value of Temperature', mean_val_temp[0][0])
mean_temp=mean_val_temp[0][0]

#now using mean_temp value to fill the nulls in temperature column
df_bristol=df_bristol.na.fill(value="9.5137689",subset=['Temperature'])

<class 'list'>
mean value of Temperature 9.513768979472026


In [ ]:
df_bristol.show()

+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5| PM2.5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|19.454|   2.9|null|20.40603|7.604697174|  9.5137689|null|        null|       AURN St Pauls|51.4628294172,-2....|2006-06-

In [ ]:
#taking the mean of the PM2.5 and than adding that average back into the column

from pyspark.sql.functions import mean

# have to rename the column pm2.5 to pm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('PM2.5', 'PM2_5')

mean_val_pm=df_bristol.select(mean(df_bristol.PM2_5)).collect()
print(type(mean_val_pm)) #mean_val_pm is a list row object
print('mean value of PM2_5', mean_val_pm[0][0])
mean_PM_2=mean_val_pm[0][0]

#now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="12.052908769",subset=['PM2_5'])
df_bristol.show()                  

<class 'list'>
mean value of PM2_5 12.052908769052758
+--------------------+--------+--------+--------+------+----+------+-----+-------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|NVPM2.5|       PM2_5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+-------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|   16.4|      19.454|   2.9|null|20.40603|7.604697174|  

In [ ]:
#taking the mean of the NVPM2.5 and than adding that average back into the column

from pyspark.sql.functions import mean

# have to rename the column NVpm2.5 to NVpm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('NVPM2.5', 'NVPM2_5')

mean_val_nvpm=df_bristol.select(mean(df_bristol.NVPM2_5)).collect()
print(type(mean_val_nvpm)) #mean_val_pm is a list row object
print('mean value of NVPM2_5', mean_val_nvpm[0][0])
mean_NVPM_2=mean_val_nvpm[0][0]

#now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="9.27125962",subset=['NVPM2_5'])
df_bristol.show()                                  

<class 'list'>
mean value of NVPM2_5 9.271259620096465
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|   NVPM2_5|       PM2_5|VPM2.5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|      16.4|      19.454|   2.9|null|20.40603|7

In [ ]:
#taking the mean of the VPM2.5 and than adding that average back into the column

from pyspark.sql.functions import mean

# have to rename the column NVpm2.5 to NVpm2_5 because has a . in it 

df_bristol = df_bristol.withColumnRenamed('VPM2.5', 'VPM2_5')
   
mean_val_vpm=df_bristol.select(mean(df_bristol.VPM2_5)).collect()
print(type(mean_val_vpm)) #mean_val_vpm is a list row object
print('mean value of VPM2_5', mean_val_vpm[0][0])
mean_VPM_2=mean_val_vpm[0][0]
                 
#now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="3.2491655",subset=['VPM2_5'])
df_bristol.show()    

<class 'list'>
mean value of VPM2_5 3.2491655692868626
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|      16.4|      19.454|      2.9|nul

In [ ]:
#taking the mean of the 03 and than adding that average back into the column

from pyspark.sql.functions import mean

   
mean_val_O3=df_bristol.select(mean(df_bristol.O3)).collect()
print(type(mean_val_O3)) #mean_val_O3 is a list row object
print('mean value of O3', mean_val_O3[0][0])
mean_val_O3=mean_val_O3[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="40.65212",subset=['O3'])
df_bristol.show()   

<class 'list'>
mean value of O3 40.652129972769764
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|  23.2|  4.6|      16.4|      19.454|      2.9|null|20

In [ ]:
#taking the mean of the NVPM10 and than adding that average back into the column

from pyspark.sql.functions import mean

   
mean_val_NVPM10=df_bristol.select(mean(df_bristol.NVPM10)).collect()
print(type(mean_val_NVPM10)) #mean_val_NVPM10 is a list row object
print('mean value of NVPM10', mean_val_NVPM10[0][0])
mean_val_NVPM10=mean_val_NVPM10[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="14.62690601",subset=['NVPM10'])
df_bristol.show()

<class 'list'>
mean value of NVPM10 14.62690601811944
+--------------------+--------+--------+--------+------+----+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|PM10|     NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+----+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|27.8|       23.2|  4.6|      16.4|      1

In [ ]:
from pyspark.sql.functions import mean

   
mean_val_PM10=df_bristol.select(mean(df_bristol.PM10)).collect()
print(type(mean_val_PM10)) #mean_val_PM10 is a list row object
print('mean value of PM10', mean_val_PM10[0][0])
mean_val_PM10=mean_val_PM10[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="22.2230",subset=['PM10'])
df_bristol.show()

<class 'list'>
mean value of PM10 22.22305344257434
+--------------------+--------+--------+--------+------+-------+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10|VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+-----+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|  4.6|      16

In [ ]:
from pyspark.sql.functions import mean

   
mean_val_VPM10=df_bristol.select(mean(df_bristol.VPM10)).collect()
print(type(mean_val_VPM10)) #mean_val_PM10 is a list row object
print('mean value of VPM10', mean_val_VPM10[0][0])
mean_val_VPM10=mean_val_VPM10[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value="3.1834",subset=['VPM10'])
df_bristol.show()

<class 'list'>
mean value of VPM10 3.183470501056342
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|  CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+----+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6|   

In [ ]:
from pyspark.sql.functions import mean

   
mean_val_CO=df_bristol.select(mean(df_bristol.CO)).collect()
print(type(mean_val_CO)) #mean_val_CO is a list row object
print('mean value of CO', mean_val_CO[0][0])
mean_val_CO=mean_val_CO[0][0]
                 
 #now using mean_temp value to fill the nulls in PM2_5 column
df_bristol=df_bristol.na.fill(value=" 0.507",subset=['CO'])
df_bristol.show()

<class 'list'>
mean value of CO 0.5077614964526624
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|           Date Time|     NOx|     NO2|      NO|SiteID|   PM10|     NVPM10| VPM10|   NVPM2_5|       PM2_5|   VPM2_5|    CO|      O3|        SO2|Temperature|  RH|Air Pressure|            Location|        geo_point_2d|           DateStart|             DateEnd|Current|     Instrument Type|
+--------------------+--------+--------+--------+------+-------+-----------+------+----------+------------+---------+------+--------+-----------+-----------+----+------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+
|2013-08-23T07:00:...|51.54044|30.50055|13.72186|   452|   27.8|       23.2|   4.6